In [5]:
fcalls = 0
function f(y,x) # in-place
    global fcalls += 1
    for i in 2:length(x)-1
    y[i] = x[i-1] - 2x[i] + x[i+1]
    end
    y[1] = -2x[1] + x[2]
    y[end] = x[end-1] - 2x[end]
    nothing
end

function g(x) # out-of-place
    global fcalls += 1
    y = zero(x)
    for i in 2:length(x)-1
    y[i] = x[i-1] - 2x[i] + x[i+1]
    end
    y[1] = -2x[1] + x[2]
    y[end] = x[end-1] - 2x[end]
    y
end

g (generic function with 1 method)

In [6]:
using SparsityDetection, SparseArrays
input = rand(30)
output = similar(input)
sparsity_pattern = jacobian_sparsity(f,output,input)
jac = Float64.(SparseArrays.sparse(sparsity_pattern))

Explored path: SparsityDetection.Path(Bool[], 1)


30×30 SparseMatrixCSC{Float64, Int64} with 88 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [7]:
using SparseDiffTools
colors = matrix_colors(jac)

30-element Vector{Int64}:
 1
 2
 3
 1
 2
 3
 1
 2
 3
 1
 2
 3
 1
 ⋮
 1
 2
 3
 1
 2
 3
 1
 2
 3
 1
 2
 3

In [8]:
using FiniteDiff
FiniteDiff.finite_difference_jacobian!(jac, f, rand(30), colorvec=colors)

In [9]:
forwarddiff_color_jacobian!(jac, f, input, colorvec = colors)

30×30 SparseMatrixCSC{Float64, Int64} with 88 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [10]:
jac

30×30 SparseMatrixCSC{Float64, Int64} with 88 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [11]:
import LinearAlgebra as la

In [12]:
function SimpleG(positions, alpha)
    res = 1
    for pos in eachcol(positions)
        res *= exp(-alpha * sum(pos.^2))
    end
    return positions .* res
end

SimpleG (generic function with 1 method)

In [13]:
import Random
rng = Random.MersenneTwister();

In [14]:
positions = (Random.rand(rng, Float64, (3, 10)) .- 0.5) .* 2

3×10 Matrix{Float64}:
  0.313355   0.962117  -0.218441  …  -0.498157   0.904421  -0.336462
  0.665702  -0.418926   0.908816     -0.227593  -0.902069   0.0107954
 -0.152059   0.921349  -0.358487      0.921291  -0.904211   0.788631

In [28]:
A = la.I(30)
A_col = SparseDiffTools.matrix_colors(A)
res = SparseDiffTools.forwarddiff_color_jacobian(positions -> SimpleG(positions, 0.5), positions, colorvec = A_col, sparsity = A)

30×30 Matrix{Float64}:
 0.00292629  0.0         0.0          0.0         …  0.0         0.0
 0.0         0.00470845  0.0          0.0            0.0         0.0
 0.0         0.0         0.000572231  0.0            0.0         0.0
 0.0         0.0         0.0          0.00620772     0.0         0.0
 0.0         0.0         0.0          0.0            0.0         0.0
 0.0         0.0         0.0          0.0         …  0.0         0.0
 0.0         0.0         0.0          0.0            0.0         0.0
 0.0         0.0         0.0          0.0            0.0         0.0
 0.0         0.0         0.0          0.0            0.0         0.0
 0.0         0.0         0.0          0.0            0.0         0.0
 0.0         0.0         0.0          0.0         …  0.0         0.0
 0.0         0.0         0.0          0.0            0.0         0.0
 0.0         0.0         0.0          0.0            0.0         0.0
 ⋮                                                ⋱              
 0.0         0

In [26]:
sum(res)

0.021167528063283047